In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.fleet.utils.fleet_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.recv import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
from shell.fleet.data.data_utilize import *
import logging
from sklearn.metrics import f1_score
import os
from shell.fleet.data.recv_utils import *
from tqdm import tqdm
import argparse
from functools import partial
from torchvision.utils import make_grid
from shell.utils.oodloss import OODSeparationLoss
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.zscore import ZSCORE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
from experiments.run import handle_combine_modes

logging.basicConfig(level=logging.INFO)

In [4]:
experiment_folder = "."
experiment_name = "combine_modes_results/modmod"
dataset = "mnist"
algo = "modular"

use_contrastive = False
num_trains_per_class = 64 if dataset != "cifar100" else 256
seed = 0
parallel = False
task_id = 5

In [5]:
def modify_cfg(net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg):
    cfg = handle_combine_modes(cfg)
    return  net_cfg, agent_cfg, train_cfg, fleet_additional_cfg, cfg

In [6]:
save_dir = get_save_dir(experiment_folder, experiment_name,
                        dataset, algo, num_trains_per_class, use_contrastive, seed)

fleet = setup_fleet(save_dir, task_id=task_id, parallel=False, modify_cfg=modify_cfg)
fleet.change_save_dir('debug_combine_modes_results')

INFO:root:Class sequence: [5 0 4 3 4 6 4 3 8 0 1 0 5 9 6 4 9 0 3 0]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)


{'train': {'component_update_freq': 100, 'num_epochs': 100, 'init_component_update_freq': 100, 'init_num_epochs': 100, 'save_freq': 10}, 'dataset': {'dataset_name': 'mnist', 'num_tasks': 10, 'num_classes_per_task': 2, 'with_replacement': True, 'num_trains_per_class': 64, 'num_vals_per_class': 50, 'remap_labels': True}, 'net': {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True}, 'sharing_strategy': {'name': 'combine_modes', 'communicator': 'modmod', 'sync_base': True, 'comm_freq': 'None', 'num_coms_per_round': 'None', 'pre_or_post_comm': 'None'}, 'seed': 0, 'algo': 'modular', 'job_name': 'mnist_modular_numtrain_64', 'num_agents': 8, 'root_save_dir': 'combine_modes_results/modmod', 'parallel': True, 'num_init_tasks': 4, 'overwrite': False, 'topology': 'fully_connected', 'edge_drop_prob': 0.0, 'agent': {'save_dir': '${root_save_dir}/${job_name}/${dataset.dataset_name}/${algo}/seed_${seed}', 'batch_size': 64, 'memory_size': 32, 'improvement_threshold': 0.

INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1992, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1954, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (2115, 1, 28, 28)
INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1901, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:Class sequence: [4 7 5 7 6 0 3 0 5 0 3 6 2 7 6 7 6 1 0 5]
INFO:root:task 0 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28) (100, 1, 28, 28) (1920, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28) (100, 1, 28, 28) (1938, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28) (100, 1, 28, 28) (1990, 1, 28, 28)
INFO:root:task 4 :(128, 1, 28, 28) (100, 1, 28, 28) (1872, 1, 28, 28)
INFO:root:task 5 :(128, 1, 28, 28) (100, 1, 28, 28) (1968, 1, 28, 28)
INFO:root:task 6 :(128

i_size 28
num_classes 2
net_cfg {'name': 'mlp', 'depth': 4, 'layer_size': 64, 'dropout': 0.5, 'no_sparse_basis': True, 'i_size': 28, 'num_classes': 2, 'num_tasks': 10, 'num_init_tasks': 4, 'use_contrastive': False}
<class 'shell.learners.er_dynamic.CompositionalDynamicER'>


INFO:root:task 6 :(128, 1, 28, 28) (100, 1, 28, 28) (1940, 1, 28, 28)
INFO:root:task 7 :(128, 1, 28, 28) (100, 1, 28, 28) (2010, 1, 28, 28)
INFO:root:task 8 :(128, 1, 28, 28) (100, 1, 28, 28) (1989, 1, 28, 28)
INFO:root:task 9 :(128, 1, 28, 28) (100, 1, 28, 28) (2145, 1, 28, 28)
INFO:root:Created fleet with 8 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized


Setting seed 69420000
Setting seed 0
Setting seed 1000
Setting seed 2000
Setting seed 3000
Setting seed 4000
Setting seed 5000
Setting seed 6000
Setting seed 7000
Setting seed 0
Setting seed 2000
Setting seed 4000
Setting seed 6000
Setting seed 8000
Setting seed 10000
Setting seed 12000
Setting seed 14000


In [7]:
fleet.agents[0].dataset.class_sequence # [1, 0]

array([7, 0, 8, 4, 8, 3, 3, 9, 8, 0, 1, 0, 5, 9, 6, 4, 9, 0, 3, 0])

In [8]:
fleet.agents[4].dataset.class_sequence # [2, 1,]

array([7, 0, 8, 4, 8, 3, 3, 9, 2, 1, 7, 6, 8, 6, 1, 8, 6, 4, 9, 8])

In [9]:
fleet.agents[1].dataset.class_sequence

array([7, 0, 8, 4, 8, 3, 3, 9, 5, 0, 3, 6, 2, 7, 6, 7, 6, 1, 0, 5])

In [10]:
agent = fleet.agents[0]

In [11]:
other = fleet.agents[1]

In [12]:
agent.train_kwargs

{'component_update_freq': 100,
 'num_epochs': 100,
 'init_component_update_freq': 100,
 'init_num_epochs': 100,
 'save_freq': 10}

In [13]:
len(other.net.components)

6

In [14]:
print(fleet.comm_freq)
print(fleet.num_coms_per_round)
print(agent.communicator)

{'modmod': 100}
{'modmod': 2}
{'modmod': <shell.fleet.mod.modmod.ModModAgent object at 0x7fef7f693fa0>}


In [15]:
 fleet.communicate(task_id=task_id, end_epoch=None, comm_freq=None, num_epochs=None,
                      final=False, strategy='modmod')

In [16]:
print(agent.net.structure[task_id])
print(other.net.structure[task_id-1])

Parameter containing:
tensor([[1.0855, 0.8372, 0.9507, 1.1122],
        [0.9681, 1.0981, 1.0678, 0.9853],
        [0.7346, 0.8820, 0.9648, 1.0160],
        [0.9683, 0.9718, 0.8731, 0.8291],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [1.0013, 0.9533, 1.0914, 1.1196]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[1.0855, 0.8372, 0.9507, 1.1122],
        [0.9681, 1.0981, 1.0678, 0.9853],
        [0.7346, 0.8820, 0.9648, 1.0160],
        [0.9683, 0.9718, 0.8731, 0.8291],
        [1.0013, 0.9533, 1.0914, 1.1196],
        [  -inf,   -inf,   -inf,   -inf]], device='cuda:0', requires_grad=True)


In [17]:
print(agent.train_kwargs['module_list'][0].bias)
print(other.net.components[-2].bias)

Parameter containing:
tensor([ 0.0452, -0.0607,  0.0722, -0.1004,  0.1251, -0.0064, -0.1164,  0.1127,
         0.0004,  0.0645,  0.0241,  0.0464,  0.1239,  0.0798,  0.0211,  0.0780,
         0.1154, -0.0562, -0.0140,  0.0268, -0.0231,  0.2038,  0.0196,  0.1246,
        -0.0479, -0.1016,  0.0194,  0.0511,  0.0079,  0.1452, -0.1037, -0.0700,
         0.0083,  0.1081,  0.0703, -0.0766,  0.1663, -0.0466,  0.0929,  0.0907,
         0.1104, -0.0234,  0.0659, -0.1202,  0.0383,  0.0312,  0.0577,  0.0238,
         0.0489, -0.0283,  0.0788,  0.0461,  0.1497,  0.0666,  0.0907,  0.0819,
        -0.0055,  0.1425, -0.1208, -0.0157, -0.0080,  0.0785,  0.0714,  0.1297],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0452, -0.0607,  0.0722, -0.1004,  0.1251, -0.0064, -0.1164,  0.1127,
         0.0004,  0.0645,  0.0241,  0.0464,  0.1239,  0.0798,  0.0211,  0.0780,
         0.1154, -0.0562, -0.0140,  0.0268, -0.0231,  0.2038,  0.0196,  0.1246,
        -0.0479, -0.1016,  0.01

In [18]:
print(agent.net.decoder[task_id].bias)
print(other.net.decoder[task_id-1].bias)

Parameter containing:
tensor([ 0.0000, -0.0381], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0959, -0.0381], device='cuda:0', requires_grad=True)


In [19]:
agent.net.structure[task_id]

Parameter containing:
tensor([[1.0855, 0.8372, 0.9507, 1.1122],
        [0.9681, 1.0981, 1.0678, 0.9853],
        [0.7346, 0.8820, 0.9648, 1.0160],
        [0.9683, 0.9718, 0.8731, 0.8291],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [1.0013, 0.9533, 1.0914, 1.1196]], device='cuda:0', requires_grad=True)

In [20]:
com = agent.communicator['modmod']

In [21]:
com.incoming_modules[1]

[{'source_task_id': 4,
  'task_sim': 0.3333333333333333,
  'module_id': 4,
  'module': Linear(in_features=64, out_features=64, bias=True),
  'decoder': Linear(in_features=64, out_features=2, bias=True),
  'structure': Parameter containing:
  tensor([[1.0855, 0.8372, 0.9507, 1.1122],
          [0.9681, 1.0981, 1.0678, 0.9853],
          [0.7346, 0.8820, 0.9648, 1.0160],
          [0.9683, 0.9718, 0.8731, 0.8291],
          [1.0013, 0.9533, 1.0914, 1.1196],
          [  -inf,   -inf,   -inf,   -inf]], device='cuda:0', requires_grad=True),
  'source_class_labels': array([5, 0])}]

In [22]:
module_list = com.get_module_list()
best, info = com.module_select.choose_best_module_from_neighbors(task_id,
                                                                                  module_list)

In [23]:
best

0

In [24]:
module_list

[{'source_task_id': 4,
  'task_sim': 0.3333333333333333,
  'module_id': 4,
  'module': Linear(in_features=64, out_features=64, bias=True),
  'decoder': Linear(in_features=64, out_features=2, bias=True),
  'structure': Parameter containing:
  tensor([[1.0855, 0.8372, 0.9507, 1.1122],
          [0.9681, 1.0981, 1.0678, 0.9853],
          [0.7346, 0.8820, 0.9648, 1.0160],
          [0.9683, 0.9718, 0.8731, 0.8291],
          [1.0013, 0.9533, 1.0914, 1.1196],
          [  -inf,   -inf,   -inf,   -inf]], device='cuda:0', requires_grad=True),
  'source_class_labels': array([5, 0]),
  'neighbor_id': 1},
 {'source_task_id': 4,
  'task_sim': 0.3333333333333333,
  'module_id': 4,
  'module': Linear(in_features=64, out_features=64, bias=True),
  'decoder': Linear(in_features=64, out_features=2, bias=True),
  'structure': Parameter containing:
  tensor([[1.4578, 0.5083, 0.6026, 0.5428],
          [0.5113, 1.4649, 1.5625, 1.1718],
          [0.4720, 0.4879, 0.5112, 1.2593],
          [0.5400, 0.511

In [25]:
module_list[best]

{'source_task_id': 4,
 'task_sim': 0.3333333333333333,
 'module_id': 4,
 'module': Linear(in_features=64, out_features=64, bias=True),
 'decoder': Linear(in_features=64, out_features=2, bias=True),
 'structure': Parameter containing:
 tensor([[1.0855, 0.8372, 0.9507, 1.1122],
         [0.9681, 1.0981, 1.0678, 0.9853],
         [0.7346, 0.8820, 0.9648, 1.0160],
         [0.9683, 0.9718, 0.8731, 0.8291],
         [1.0013, 0.9533, 1.0914, 1.1196],
         [  -inf,   -inf,   -inf,   -inf]], device='cuda:0', requires_grad=True),
 'source_class_labels': array([5, 0]),
 'neighbor_id': 1}

In [26]:
com.train_kwargs

{'component_update_freq': 100,
 'num_epochs': 100,
 'init_component_update_freq': 100,
 'init_num_epochs': 100,
 'save_freq': 10,
 'module_list': [Linear(in_features=64, out_features=64, bias=True)],
 'train_candidate_module': True}

In [27]:
# task_id=5, mnist seed 0, agent 2, other 6, task_sim 1

In [28]:
agent.train_kwargs

{'component_update_freq': 100,
 'num_epochs': 100,
 'init_component_update_freq': 100,
 'init_num_epochs': 100,
 'save_freq': 10,
 'module_list': [Linear(in_features=64, out_features=64, bias=True)],
 'train_candidate_module': True}